In [ ]:
# import public packages
import os
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.patches import Rectangle
import scipy
import mne
import sys

from mne.time_frequency import tfr_morlet
from mne.baseline import rescale
from scipy.signal import spectrogram, hann, butter, filtfilt, hilbert
from scipy import signal, interpolate, stats
from scipy.interpolate import make_interp_spline, BSpline
from io import open
from importlib import reload

# import own functions
from utils import find_folders
import dat_preproc
import fix_annot_onsets
import mat2fif
import baseline_correction
import normalization
import power_spectrum

### Add Directories/ Load Files

In [ ]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

In [ ]:
percept_ID = 'sub050'
fname = 'sub-20220413PStn_ses-2023053101280985_run-BrainSense20230531022500.mat'

raw = mne.io.read_raw_fieldtrip(
    os.path.join(
        project_path,
        'data',
        'raw_data',
        'without_med_FTG',
        fname
    ),
    info = None
)

subID = 'Sub050'

In [ ]:
raw.ch_names

In [ ]:
raw = mne.io.read_raw_fif(os.path.join(
    project_path,
        'data',
        'Fifs',
        'without_med_FTG',
        'Sub050_FIF.fif'
    )
)

subID = 'Sub050'

In [ ]:
reload(dat_preproc)
%matplotlib qt
matplotlib.rc('font', size=10)
x = raw.get_data(reject_by_annotation = 'omit',picks=[0,1])
raw = raw
win_samp = 250
noverlap = 0.5
window = hann(win_samp, sym=False)
f, t, Sxx = dat_preproc.fft_rawviz(raw, x, win_samp, noverlap)

In [ ]:
fft_name = str(subID)+'_FFT'
print(fft_name)

In [ ]:
#Saving the figures & files
fft_fig = os.path.join(project_path, 'figures','FFts','without_med_FTG/')
fft_file = os.path.join(project_path, 'data','FFTs','without_med_FTG/')

plt.savefig(str(fft_fig)+str(fft_name),dpi = 300)

np.save(str(fft_file)+str(fft_name)+'.npy',Sxx)

In [ ]:
#Saving the raw data as a fif file
raw2 = raw.copy()
Draw = raw2.get_data()
info = raw.info
new_raw = mne.io.RawArray(Draw, info)

fif_name = os.path.join(project_path, 'data','Fifs','without_med_FTG/',str(subID)+'_FIF.fif')
new_raw.save(fif_name, overwrite = True)


In [ ]:
#Zscore Normalization of raw data & plot of new figure
reload(baseline_correction)
data = Sxx
t = t
baseline = (None, None)
raw = raw
stim_ch = 1
bs_data = baseline_correction.baseline_corr(data, t, baseline, raw = raw, stim_ch = 4)

In [ ]:
fft_fig = os.path.join(project_path, 'figures','FFts','test/')
fft_file = os.path.join(project_path, 'data','FFTs','test/')

plt.savefig(str(fft_fig)+str(fft_name)+'_zscored',dpi = 300)
np.save(str(fft_file)+str(fft_name)+'_zscored'+'.npy',bs_data)

In [ ]:
d = {'onset': [1, 500, 778],
    'duration': [30, 30, 10],
    'description': ['StimOff','StimOn','Rebound']
}

epoch_df = pd.DataFrame(data = d)
print(epoch_df)

In [ ]:
## Print Power Spectra with Raw Data
reload(dat_preproc)
filt_dat = raw.get_data()
time_onsets = epoch_df
window = 250
noverlap = 0.5
ylim2 = 0.4

fig = plt.figure(figsize = (12,5))

plt.subplot(1,2,1)
title1 = str(subID)+ '_LSTN'
ps1 = dat_preproc.epoch_ps(filt_dat, time_onsets, window, noverlap, 0, 0.3, title1)

plt.subplot(1,2,2)
title2 = str(subID)+ '_RSTN'
ps2 = dat_preproc.epoch_ps(filt_dat, time_onsets, window, noverlap, 1, 0.2, title2)

plt.show()

In [ ]:
## Printing the Power Spectra (RAW)
ps_all = np.concatenate((ps1,ps2), axis = 0)
ps_df = pd.DataFrame(np.transpose(ps_all),
    columns = ['StimOff_LSTN','StimOn_LSTN','Rebound_LSTN',
    'StimOff_RSTN','StimOn_RSTN','Rebound_RSTN'],
    )
print(ps_df)

In [ ]:
#Saving the figures & files
fft_fig = os.path.join(project_path, 'figures','Power_Spectra','without_med_FTG/')
fft_file = os.path.join(project_path, 'data','power_spectra','without_med_FTG/')

plt.savefig(str(fft_fig)+str(fft_name)+'_PS',dpi = 150)
ps_df.to_csv(str(fft_file)+str(fft_name)+'_PS.csv')

In [ ]:
## Print Power Spectra with Zscored Data
fig, axes = plt.subplots(1,1, figsize = (8,6))

side = 0

ps_m1s0 = np.mean(bs_data[side,:,epoch_df.onset[0]:epoch_df.onset[0]+ epoch_df.duration[0]],1)
ps_m1s1 = np.mean(bs_data[side,:,epoch_df.onset[1]:epoch_df.onset[1]+ epoch_df.duration[1]],1)
ps_reb = np.mean(bs_data[side,:,epoch_df.onset[2]:epoch_df.onset[2]+ epoch_df.duration[2]],1)

plt.plot(np.arange(1,127),ps_m1s0,label = epoch_df.description[0])
plt.plot(np.arange(1,127),ps_m1s1,label = epoch_df.description[1])
plt.plot(np.arange(1,127),ps_reb,label = epoch_df.description[2])
    
plt.xlim(60,90)
plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power (z-scored)')
    
plt.legend()
plt.show()


In [ ]:
ps_allzscored_df = pd.DataFrame(
    {
        'Spontan': ps_m1s0,
        'StimOn': ps_m1s1,
        'Rebound': ps_reb
    }
)
print(ps_allzscored_df)

In [ ]:
plt.savefig(str(fft_fig)+str(fft_name)+'_Contra_zscoredPS',dpi = 150)
ps_allzscored_df.to_csv(str(fft_file)+str(fft_name)+'_Contra_zscoredPS.csv')

### Checking Spectral Width

In [ ]:
#Import the power spectrum csv
ps_tbl = pd.read_csv(os.path.join(
    project_path,
        'data',
        'power_spectra',
        'test',
        'Sub045_FFT_PS.csv'
))

subID = 'Sub045'

x1 = ps_tbl.StimOff_LSTN[50:101]
x2 = ps_tbl.StimOn_LSTN[50:101]
plt.plot(np.arange(0,51),x1)
plt.plot(np.arange(0,51),x2)
plt.xticks(np.arange(0,51,10), labels = np.arange(50,101,10))

plt.ylim(0,0.4)

In [ ]:
reload(power_spectrum)
%matplotlib qt

height = 0.04

peaks1, results_half1, peaks2, results_half2 = power_spectrum.powerSpectrum_width(x1, x2, height, subID)
print(results_half1)
print(results_half2)

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'figures',
    'Power_Spectra',
    'test',
    str(subID)+'_PSWidth'
),
dpi = 150)

### Average all Power Spectra and Plot them

In [ ]:
## Interpolate and plot power spectrum
trial_ps = pd.read_csv(os.path.join(
    project_path,
        'data',
        'power_spectra',
        'test',
        'Sub005_FFT_PS.csv'
)
)

print(trial_ps)

In [ ]:
reload(dat_preproc)
dat_preproc.mypower(trial_ps['StimOff_RSTN'])
dat_preproc.mypower(trial_ps['StimOn_RSTN'])
plt.xlim(50,100)

In [ ]:
os.path.join(
   project_path,
   'data', 'power_spectra', 'test'
)

In [ ]:
print(filename)

In [ ]:
#loop through all csv files in ZSCORED
from scipy import stats
all_spontan = []
all_subh = []

ps_path = os.path.join(
   project_path,
   'data', 'power_spectra', 'without_med_FTG'
)

'''
for filename in os.listdir(ps_path):
   if filename.endswith('FFT_Ipsi_zscoredPS.csv'):
   #with open(os.path.join(ps_path, filename), 'r') as f: # open in readonly mode
      this_df = pd.read_csv(os.path.join(ps_path,filename))
      this_spontan = this_df['Spontan']
      this_subh = this_df['StimOn']


      all_spontan.append(this_spontan)
      all_subh.append(this_subh)

'''

for filename in os.listdir(ps_path):
   if filename.endswith('FFT_PS.csv'):
      if filename.startswith('Sub007') or filename.startswith('Sub050'):
   #with open(os.path.join(ps_path, filename), 'r') as f: # open in readonly mode
         this_df = pd.read_csv(os.path.join(ps_path,filename))
         this_spontan = this_df['StimOff_LSTN']
         this_subh = this_df['StimOn_LSTN']
      else: 
         this_df = pd.read_csv(os.path.join(ps_path,filename))
         this_spontan = this_df['StimOff_RSTN']
         this_subh = this_df['StimOn_RSTN']

   all_spontan.append(this_spontan)
   all_subh.append(this_subh)


In [ ]:
all_spontan_df = pd.DataFrame(np.transpose(all_spontan))
all_subh_df = pd.DataFrame(np.transpose(all_subh))

In [ ]:
reload(dat_preproc)
dat_preproc.mypower(all_spontan_df)
plt.xlim(50,100)

In [ ]:
%matplotlib qt
plt.plot(all_spontan_df,color = '#1f77b4',lw = 1, alpha = 1, label = 'Med On-Stim Off')
plt.plot(all_subh_df,color = 'red',lw = 1, alpha = 1, label = 'Med On-Stim On')

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys())

plt.xlim(50,90)
plt.ylim(0,0.25)
plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power')

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'figures',
    'Power_Spectra',
    'without_med_FTG',
    'AllSub_Ipsi_PowerSpectra'
),
dpi = 200)

In [ ]:
all_psSpontan = pd.read_csv(os.path.join(
    project_path,
        'results',
        'AllSpontan_PsZscored.csv'
))


all_psSubharm = pd.read_csv(os.path.join(
    project_path,
        'results',
        'AllSubharm_PsZscored.csv'
))

In [ ]:
all_psSpontan = all_psSpontan.drop('Sub029', axis=1)

In [ ]:
all_ps_mean1 = np.mean(all_psSpontan,1)
all_ps_sem1 = stats.sem(all_psSpontan,1)

all_ps_mean2 = np.mean(all_psSubharm,1)
all_ps_sem2 = stats.sem(all_psSubharm,1)

In [ ]:
plt.plot(np.arange(1,127), all_ps_mean1, label = 'Spontaneous FTG')
plt.fill_between(np.arange(1,127), all_ps_mean1-all_ps_sem1, all_ps_mean1+all_ps_sem1, alpha = 0.2)

plt.plot(np.arange(1,127), all_ps_mean2, label = 'DBS-Induced FTG')
plt.fill_between(np.arange(1,127), all_ps_mean2-all_ps_sem2, all_ps_mean2+all_ps_sem2, alpha = 0.2)

plt.xlim(50, 100)
plt.ylim(-0.4, 2)
plt.legend()

plt.xlabel('Frequency [Hz]')
plt.ylabel('LFP Power [z-scored]')
plt.title('Averaged Power Spectra [N = 6]')

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'results',
    'Avg_PS'
),
dpi = 200)

In [ ]:
###Compare Power Spectra Widths

widths_tbl = pd.read_csv(os.path.join(
    project_path,
    'results',
    'PS_Widths.csv'
))
print(widths_tbl)

In [ ]:
import mlxtend
import seaborn as sns

In [ ]:
widths_all = np.concatenate((widths_tbl['WidthMed'],widths_tbl['WidthSubh']), axis = 0)
val_all = np.concatenate((np.repeat(0,7), np.repeat(1,7)), axis = 0)
widths_dict = {
    'widths': widths_all,
    'conds': val_all,
    'paired':[1,2,3,4,5,6,7,1,2,3,4,5,6,7]
}

df1=pd.DataFrame.from_dict(widths_dict)
print(df1)

In [ ]:
%matplotlib qt
ax = sns.boxplot(x='conds',y='widths',data=df1, boxprops=dict(alpha=.8), width = 0.3)
sns.stripplot(data=df1, x='conds',y='widths', jitter = 0, size = 10, edgecolor = 'black')
sns.lineplot(
    data=df1, x='conds',y='widths', units = 'paired',
    color=".7", estimator=None, 
    linestyle=':'
)
ax.set_xticklabels({'M1S0','M1S1'})
ax.set_xlabel('')
ax.set_ylabel('Widths [Freq Bins]')
plt.text(0.9,2.4,'p = .015')
plt.rcParams.update({'font.size': 15})

In [ ]:
p_value = mlxtend.permutation_test(
    widths_tbl['WidthMed'], widths_tbl['WidthSubh'], paired=True, method="approximate", seed=0, num_rounds=100000
)

print(p_value)

In [ ]:
np.round(p_value,decimals = 3)

In [ ]:
plt.savefig(os.path.join(
    project_path,
    'results',
    'PS_Widths'
),
dpi = 200)

In [ ]:
cdrs_df = pd.read_excel(os.path.join(
    project_path,
    'results',
    'Trial_CDRSAll.xlsx'
))
print(cdrs_df)

In [ ]:
score1 = cdrs_df.loc[(cdrs_df['Laterality'] == 'Total') & (cdrs_df['Cond'] == 1),"cdrs_score"]
score2 = cdrs_df.loc[(cdrs_df['Laterality'] == 'Total') & (cdrs_df['Cond'] == 3),"cdrs_score"]

p_value = mlxtend.permutation_test(
    np.array(score1), np.array(score2), paired=True, method="approximate", seed=0, num_rounds=100000
)

print(p_value)

In [ ]:
%matplotlib qt
sns.set_theme(style="ticks", palette="pastel")
ax = sns.boxplot(x='Cond',y='cdrs_score',hue = 'Laterality', palette = ['m','g'], data=cdrs_df, width = 0.7)
sns.stripplot(data=cdrs_df, x='Cond',y='cdrs_score',hue = 'Laterality', jitter = False, size = 10, dodge = True, palette = ['#7E1E9C','#054907'])
sns.lineplot(
    data=cdrs_df, x=cdrs_df['Cond']-1, y='cdrs_score', units= 'SubCode', hue = 'Laterality',
    color=".3",
    palette=['m','g'],
    linestyle=':', estimator = None
)
handles, labels = ax.get_legend_handles_labels()
l = plt.legend(handles[0:2], labels[0:2])
ax.set_ylim(-2,15)

ax.set_xlabel('')
ax.set_xticklabels(['M1S0','M1S1-PreSub','M1S1-Sub'])
ax.set_ylabel('CDRS Score')


In [ ]:
plt.savefig(os.path.join(
    project_path,
    'results',
    'CDRS_boxplot'
),
dpi = 200)